In [ ]:
# pip install wordcloud
# pip install textblob
!pip install emoji
!pip install fasttext

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path="For the files folder"

In [ ]:
# Python Libs
import re
import csv
import os
import datetime
import emoji
import pandas as pd
import numpy as np
import time

# plotting and math libs
from tqdm import tqdm
import matplotlib.pylab
import matplotlib.pyplot as plt
import seaborn as sns

import tweepy
from tweepy import OAuthHandler
import operator 
import json
from collections import Counter
from collections import defaultdict
import datetime as datetime
import requests

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Political Tweets Collection

Authentication section

In [ ]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

auth = OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_secret)
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

### Scrapping Methods - trend tweets from twitter 

Some helper function for cleaning the tweets

In [ ]:
def clean_tweet(tweet):
    tweet = tweet.lower()
    regs = [
        r'(\/\/www[^\s]+)',
        r'(pic.twitter.com\/[^\s]+)',
        r'(?i)\b((?:https?:\/\/[^\s]+))',
        r'https'
    ]
    prev = tweet
    for reg in regs:
        prev = re.sub(reg, " ", prev)
    return re.sub("\n", " ", prev)

def get_tweets_list(account):
    tweets = []
    with open(f'./data/{account}.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        keys = next(csv_reader)
        for row in csv_reader:
            tweet = {}
            tweet[keys[0]] = row[0]
            tweet[keys[1]] = row[1]
            tweet[keys[6]] = clean_tweet(row[6])
            tweets.append(tweet)
    return tweets

def get_tweets(username):
    l = get_tweets_list(username)
    l.reverse()
    return l

def give_emoji_free_text(text): 
    return emoji.get_emoji_regexp().sub(r'', text)

In [ ]:
# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
#     percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / total))

    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

Searching tweets according to a search words - 1 week scrapping tweets only

In [ ]:
def build_json_tweets(search_words, file_path, n_count=200_000, since=datetime.datetime(2021, 1, 1, 0, 0, 0)):
    tweet_max_counter = 0
    max_tweets = 10_000
    for tweet in tweepy.Cursor(api.search,
                       q=search_words,
                       lang="he",
                       count=n_count,
                       tweet_mode='extended',
                       ).items():
        if tweet.created_at > since:
            
            if (tweet_max_counter/max_tweets*100 < 100):
                printProgressBar(tweet_max_counter, max_tweets, prefix = 'Progress:', suffix = 'Complete', length = 50)
                tweet_max_counter += 1

            data = {"tweet_id": str(tweet.id),
                "created_at": tweet.created_at.__str__(),
                "source": give_emoji_free_text(tweet.source),
                "text": clean_tweet(give_emoji_free_text(tweet.full_text)),
                "retweet_count": tweet.retweet_count,
                "user_id": tweet.user.id,
                "user_name": give_emoji_free_text(tweet.user.name),
                "user_followers_count": tweet.user.followers_count,
                "user_friends_count": tweet.user.friends_count,
                "user_statuses_count": tweet.user.statuses_count,
                "hashtags": tweet.entities['hashtags'],
    #             "user_mentions": tweet.entities['user_mentions'],
                "reply_to_status_id_str": tweet.in_reply_to_status_id_str,
                "reply_to_user_id": tweet.in_reply_to_user_id
               }
            with open(file_path, 'a', encoding="utf-8") as f:
                json.dump(data, f, ensure_ascii=False)
                f.write("\n")
    print(f' \n Scrapped total of {tweet_max_counter} tweets')

Loading json format of the tweets into a list of dictionaries

In [ ]:
def extract_json_tweets_into_list(file_path):
    tweets = []
    with open(file_path, 'r', encoding="utf-8") as f:
        for line in f:
            res = json.loads(line)
#             for k, v in res.items():
#                 if v is str:
#                     res[k] = v.encode("latin_1").decode("utf_8")
            tweets.append(res)
    return tweets

More helper functions

In [ ]:
def sort_list_by_date(t_list):
    return sorted(t_list, key = lambda x: x['created_at'])
def sort_list_by_followers(t_list):
    return sorted(t_list, key = lambda x: x['user_followers_count'], reverse=True)
def get_users_list(users_path):
    my_file = open(users_path, "r")
    content_list = my_file.readlines()
    content_list = [a.split("\n")[0] for a in content_list]
    return content_list

Retrieving user's tweets from his timeline since January 2021

In [ ]:
def extract_user_tweets(target, file_path, political_stance, since=datetime.datetime(2021, 1, 1, 0, 0, 0)):
    """
    Extracting user tweets and classify them with the political stance according to their
    political positin as right wind or left wing.
    @param: target: (str) user screen name
    @param: file_path: (str) the path to write the json to
    @param: political_stance: (int) 0 for right wing and 1 for left wing
    @param: since: (date) start from January 2021 
    """
    item = api.get_user(target)
    for tweet in tweepy.Cursor(api.user_timeline, id=target, tweet_mode='extended').items():
        if tweet.created_at > since:
            data = {"tweet_id": str(tweet.id),
                    "created_at": tweet.created_at.__str__(),
                    "source": give_emoji_free_text(tweet.source),
                    "text": clean_tweet(give_emoji_free_text(tweet.full_text)),
                    "retweet_count ": tweet.retweet_count, 
                    "user_id": tweet.user.id,
                    "user_name": give_emoji_free_text(tweet.user.name),
                    "user_followers_count": tweet.user.followers_count,
                    "user_friends_count": tweet.user.friends_count,
                    "user_statuses_count": tweet.user.statuses_count,
                    "hashtags": tweet.entities['hashtags'],
        #             "user_mentions": tweet.entities['user_mentions'],
                    "reply_to_status_id_str": tweet.in_reply_to_status_id_str,
                    "reply_to_user_id": tweet.in_reply_to_user_id,
                    "class": political_stance
                       }
            with open(file_path, 'a', encoding="utf-8") as f:
                json.dump(data, f, ensure_ascii=False)
                f.write("\n")

In [ ]:
def write_tweets_into_csv(allTweetsPath, file_path, data, with_header=False):
    columns = list(data[0].keys())
    if os.path.exists(allTweetsPath):
        ids_list = pd.read_csv(allTweetsPath, usecols=['tweet_id'])['tweet_id'].tolist()
    else:
        ids_list = []
    with open(allTweetsPath, 'a', encoding = "utf-8", newline='') as f:
        writer = csv.DictWriter(f, fieldnames=columns)
        if with_header:
            writer.writeheader()
        for record in data:
            if record['tweet_id'] in ids_list:
                nothing=""
            else:
                if not isinstance(record['tweet_id'], str):
                    record['tweet_id'] = str(record['tweet_id'])
                writer.writerow(record)

In [ ]:
def extract_followers_from_user(target, users_file):
    followers_threshold = 10000
    max_followers = 10
    current_count = 0
    users_list = get_users_list(users_file)
    followers_list = []
    followers = tweepy.Cursor(api.followers, screen_name=target).items()
    
    for follower in followers:
        if follower.followers_count > followers_threshold and follower.screen_name not in users_list and current_count < max_followers:
            followers_list.append(follower.screen_name)
            if (current_count/max_followers*100 < 100):
                printProgressBar(current_count, max_followers, prefix = 'Progress:', suffix = 'Complete', length = 50)
                current_count += 1
    
    out = open(users_file, 'a')
    for user_name in followers_list:
        out.write(user_name + "\n")
    out.close()

In [ ]:
def sort_and_write(allTweetsPath, file_path, data, with_header=False, sort_by='date'):
    """
    Joining the sorted data into the collected tweets csv file.
        @param: allTweetsPath: (str) path of the collected tweets csv file
        @param: file_path: (str) path for the json to read from - dont use unless you provide the data
        @param: data: (list<dict>) the tweets to write into the csv file
        @param: with_header: (bool) write the columns names of the data - sont use unless it is your first writing
        @param: sort_by: (str) 2 Options of sorting: date or followers
    """
    sorted_tweets = []
    if sort_by == 'date':
        sorted_tweets = sort_list_by_date(data)
    else:
        sorted_tweets = sort_list_by_followers(data)
    
    write_tweets_into_csv(allTweetsPath, file_path, sorted_tweets, with_header)

In [ ]:
def read_and_get_users_tweets(allUsersTweetsPath, users_list_path, files_dir, users_political_stance, with_header=False, sort_by='date'):
    """
    Retrieiving the user tweets, writing them into seperate json and then writing them into the collected tweets csv file.
        @param: allUsersTweetsPath: (str) path of the collected tweets csv file
        @param: users_list_path: (str) path for the file to read from the users screen names
        @param: files_dir: (str) path for the dir to write the users tweets as json
        @param: users_political_stance: (int) 0 for right wing and 1 for left wing
        @param: with_header: (bool) write the columns names of the data - sont use unless it is your first writing
        @param: sort_by: (str) 2 Options of sorting: date or followers
    """
    
    users_list = get_users_list(users_list_path)
    user_count = 0
    max_users = len(users_list)
    backoff_counter = 1 # for extending the host connection 
    runnig_idx = 0
    fail_counter = 0
    for i in range(len(users_list)):
        try:
            while True:
                target = users_list[i]
                
                if (user_count/max_users*100 < 100):
                        printProgressBar(user_count, max_users, prefix = 'Progress:', suffix = 'Complete', length = 50)
                        user_count += 1

                file_path = f'{files_dir}\\{target}_user.json'
                extract_user_tweets(target, file_path, users_political_stance)
#                 if os.path.exists(file_path):
#                     data = extract_json_tweets_into_list(file_path)
#                     sort_and_write(allTweetsPath, file_path, data, with_header, sort_by)
                runnig_idx += 1
                break
        except tweepy.TweepError as e:
            print(f'{backoff_counter}, {target}: {e.reason}')
            time.sleep(2*backoff_counter)
            if backoff_counter < 120:
                backoff_counter += 1
                
            try:
                
                ### format of the reason is str like this "[..., 'key': value, ...]"
                reason = re.sub(r"[\[\]]", "", e.reason)
                reason = re.sub("[\']", "\"", reason)
                reason = json.loads(reason)
                
                ### User not found = code 50
                if reason['code'] == 50:
                    continue
            except:
                ### if the same user is failing more than 3 times than move on to the next user
                if fail_counter < 2 and target == users_list[i]:
                    fail_counter += 1
                elif target == users_list[i]:
                    fail_counter = 0
                    if i < len(users_list)-1:
                        target = users_list[i+1]
                        continue
                    else:
                        i = len(users_list)+1
                        break
            i -= 1
            continue

In [ ]:
def read_jsons_and_write_to_csv(users_tweets_path, dir_path, to_write=True):
    """
    Read the dir path of the jsons for each user's tweets and join them together into a csv file
    @param: users_tweets_path: (str) path for the csv file
    @param: dir_path: (str) path for the jsons of the user's tweets
    """
    dir_list = os.listdir(dir_path)
    data = []
    
    for name in dir_list:
        fp = f'{dir_path}\\{name}'
        with open(fp, 'r') as f:
            data.extend([d for d in sort_list_by_date(extract_json_tweets_into_list(fp))])
    
    keys = list(data[0].keys())
    df = pd.DataFrame(data, columns=keys)
    
    df.drop_duplicates(subset=['tweet_id'], inplace=True)
    
    if to_write:
        df.to_csv(users_tweets_path, columns=keys, index=False)
    
    return df

## User Tweets

#### Right wing users

In [ ]:
allTweetsPath = 'F:\\SocialDynamics\\TrendsProject\\Tweets\\User_Tweets.csv'
# users_list_path = 'F:\\SocialDynamics\\TrendsProject\\RightWingUsers.txt'
users_list_path = 'F:\\SocialDynamics\\TrendsProject\\RightComplete.txt'
files_dir = 'F:\\SocialDynamics\\TrendsProject\\Tweets\\UsersTweets'
read_and_get_users_tweets(allTweetsPath, users_list_path, files_dir, users_political_stance=0, with_header=False, sort_by='date')

In [ ]:
dir_path = 'F:\\SocialDynamics\\TrendsProject\\Tweets\\RightUsersTweets'
users_tweets_path = 'F:\\SocialDynamics\\TrendsProject\\Tweets\\Right_Users_Tweets.csv'
read_jsons_and_write_to_csv(users_tweets_path, dir_path)

#### Left wing users

In [ ]:
allTweetsPath = 'F:\\SocialDynamics\\TrendsProject\\Tweets\\User_Tweets.csv'
users_list_path = 'F:\\SocialDynamics\\TrendsProject\\LeftWingUsers.txt'
files_dir = 'F:\\SocialDynamics\\TrendsProject\\Tweets\\LeftUsersTweets'
read_and_get_users_tweets(allTweetsPath, users_list_path, files_dir, users_political_stance=1, with_header=False, sort_by='date')

1, avi07325462: Failed to send request: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
2, falach_roy: Failed to send request: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
3, ron_zo2020: [{'code': 50, 'message': 'User not found.'}]--| 37.4% Complete
4, EilonTohar: Failed to send request: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
5, gutweind: Failed to send request: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
6, ajove1: Failed to send request: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
7, GuetItai: Failed to send request: ('Connecti

In [ ]:
dir_path = 'F:\\SocialDynamics\\TrendsProject\\Tweets\\LeftUsersTweets'
users_tweets_path = 'F:\\SocialDynamics\\TrendsProject\\Tweets\\Left_Users_Tweets.csv'
read_jsons_and_write_to_csv(users_tweets_path, dir_path)

,tweet_id,created_at,source,text,retweet_count,user_id,user_name,user_followers_count,user_friends_count,user_statuses_count,hashtags,reply_to_status_id_str,reply_to_user_id,class
0,1344916630665637888,2021-01-01 08:01:26,Twitter Web App,שיתופי פעולה כאלה ראינו לאחרונה בתחום האלימות ...,0,2960946733,ניצן כהנא Nitzan Kahana,5599,833,8333,[],1344916628845293569,2.960947e+09,1
1,1344916628845293569,2021-01-01 08:01:26,Twitter Web App,שאל אותי @kobisudriאיך גורמים לכנסת לקדם מאבקי...,1,2960946733,ניצן כהנא Nitzan Kahana,5599,833,8333,[],None,NaN,1
2,1344920004362432513,2021-01-01 08:14:51,Twitter Web App,@kobisudri @shuli_mr @zehavagalon @aliza_lavie...,0,2960946733,ניצן כהנא Nitzan Kahana,5599,833,8333,[],1344918837339283457,7.113163e+08,1
3,1344932236357996545,2021-01-01 09:03:27,Twitter for Android,@kobisudri @avarplyly מקדמי החוק הכי לוהטים הם...,0,2960946733,ניצן כהנא Nitzan Kahana,5599,833,8333,[],1344926009846591489,7.113163e+08,1
4,1344933790783860737,2021-01-01 09:09:38,Twitter for Android,@kobisudri @avarplyly הניסוי הזה נכשל כשלון חר...,0,2960946733,ניצן כהנא Nitzan Kahana,5599,833,8333,[],1344932876752707584,7.113163e+08,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395133,1414439555432861699,2021-07-12 04:20:43,Twitter Web App,6/ מתינו מוטלים לפנינו. דמיהם זועקים אלינו מן ...,2,884063996,זהבה גלאון,119261,2191,10157,[],1414439554141020163,8.840640e+08,1
395134,1414439554141020163,2021-07-12 04:20:43,Twitter Web App,5/ מדובר בחיי אדם. רק בעקבות שתי עתירות שהגשתי...,0,884063996,זהבה גלאון,119261,2191,10157,[],1414439552803016707,8.840640e+08,1
395135,1414439552803016707,2021-07-12 04:20:43,Twitter Web App,"4/ לאחרונה הגיש ח""כ @yairgolan1 הצעת חוק שניסח...",0,884063996,זהבה גלאון,119261,2191,10157,[],1414439551402139649,8.840640e+08,1
395136,1414626963419959301,2021-07-12 16:45:25,Twitter Web App,"מה שמדהים לא פחות מהעסקאות המפוקפות שעשה קק""ל ...",2,884063996,זהבה גלאון,119261,2191,10157,[],None,NaN,1


## Collecting Trends

Reading the trends we have collected from txt file 

In [ ]:
trends_text = []
with open("F:\\SocialDynamics\\TrendsProject\\trends.txt", 'r', encoding='utf8') as f:
    trends_text = f.read().splitlines()
    
trends = []
for t in trends_text:
    t = t.split('\t')
    if len(t) < 3:
        trends.append(t[1])
    else:
        trends.append(t[2])
len(trends), trends[:5]

(50, ['שאשא פאקה ', 'חוק צרפתי', 'של שרה', 'עם שרה', 'רק ביבי'])

In [ ]:
trends = [x.strip(' ') for x in trends]
trends

['שאשא פאקה',
 'חוק צרפתי',
 'של שרה',
 'עם שרה',
 'רק ביבי',
 'חוזרים_לחיים',
 'כחול לבן',
 'רק נתניהו',
 'רק מחל',
 'בן גביר',
 'יאיר לפיד',
 'ביבי_ברח',
 'תחת לפיד',
 'העם אמר',
 'תם עידן',
 'שר משפטים',
 'נבצרות עכשיו',
 'בחירות חמישיות',
 'יאיר גולן',
 'עד מדינה',
 'יוסי כהן',
 'של הפרקליטות',
 'מנסור עבאס',
 'רע"מ',
 'עוד בחירות',
 'חאן אל',
 'תם עידן הציונות',
 'של שיקלי',
 'עמיחי שיקלי',
 'צבע אדום',
 'יום ירושלים שמח',
 'מורשת נתניהו',
 'בגלל בלה חדיד',
 'מירב מיכאלי',
 'של סלקום',
 'אבי גבאי',
 'מרים פרץ',
 'מר ביטחון',
 'חוק יסוד',
 'עלה בידי',
 'עלה בידו',
 'תודה בני',
 'ילדה קטנה',
 'למה_גדעון_התפטר_ב2014',
 'למה גדעון סהר',
 'למה סהר',
 'של תא כפפות',
 'נוכל שקרן',
 'נאור יחיא',
 'נתניהו מאוחדת']

In [ ]:
def extract_tweets_from_trends(trends, dir_path):
    for t in trends:
        t_path = f'{dir_path}/{t}.json'
        build_json_tweets(t, t_path)

In [ ]:
dir_path = 'F:\\SocialDynamics\\TrendsProject\\Tweets\\TrendsTweets'
extract_tweets_from_trends(trends[24:], dir_path)

 rogress: |█-------------------------------------------------| 2.9% Complete
 Scrapped total of 288 tweets
 rogress: |█████████████████---------------------------------| 35.2% Complete
 Scrapped total of 3521 tweets
 
 Scrapped total of 0 tweets
 rogress: |███-----------------------------------------------| 7.2% Complete
 Scrapped total of 723 tweets
 rogress: |█████---------------------------------------------| 11.1% Complete
 Scrapped total of 1107 tweets
 rogress: |--------------------------------------------------| 0.9% Complete
 Scrapped total of 95 tweets
 rogress: |--------------------------------------------------| 0.0% Complete
 Scrapped total of 2 tweets
 rogress: |--------------------------------------------------| 1.2% Complete
 Scrapped total of 119 tweets
 rogress: |--------------------------------------------------| 0.0% Complete
 Scrapped total of 2 tweets
 rogress: |██------------------------------------------------| 5.5% Complete
 Scrapped total of 549 tweets
 rogress

In [ ]:
trends_tweets_path = 'F:\\SocialDynamics\\TrendsProject\\Tweets\\Trends_Tweets.csv'
read_jsons_and_write_to_csv(trends_tweets_path, dir_path)

,tweet_id,created_at,source,text,retweet_count,user_id,user_name,user_followers_count,user_friends_count,user_statuses_count,hashtags,reply_to_status_id_str,reply_to_user_id
0,1409768853308162052,2021-06-29 07:01:01,Twitter Web App,חברים : כל אחד שמצייץ .. נא להוסיף בסוף הציוץ ...,19.0,892778092200067072,להפסיק לבכות ולהתחיל להלחם - הייטקיסט בן 38 ימני,1920,3423,9522,"[{'text': 'למה_גדעון_התפטר_ב2014', 'indices': ...",None,NaN
1,1409769351385956352,2021-06-29 07:03:00,Twitter Web App,בוקר טוב לכולם !! #למה_גדעון_התפטר_ב2014,3.0,892778092200067072,להפסיק לבכות ולהתחיל להלחם - הייטקיסט בן 38 ימני,1920,3423,9522,"[{'text': 'למה_גדעון_התפטר_ב2014', 'indices': ...",None,NaN
2,1409769893176676352,2021-06-29 07:05:09,Twitter for iPhone,rt @bibist_hitech: בוקר טוב לכולם !! #למה_גדעו...,3.0,340325107,Yoav Frank האמת תנצח,3088,4140,15241,"[{'text': 'למה_גדעון_התפטר_ב2014', 'indices': ...",None,NaN
3,1409769934373150722,2021-06-29 07:05:19,Twitter for iPhone,rt @bibist_hitech: חברים : כל אחד שמצייץ .. נא...,19.0,340325107,Yoav Frank האמת תנצח,3088,4140,15241,"[{'text': 'למה_גדעון_התפטר_ב2014', 'indices': ...",None,NaN
4,1409770384556281859,2021-06-29 07:07:06,Twitter for iPhone,rt @bibist_hitech: חברים : כל אחד שמצייץ .. נא...,19.0,1105767815968108545,Edencohen,381,401,32312,"[{'text': 'למה_גדעון_התפטר_ב2014', 'indices': ...",None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53523,1418439023962566657,2021-07-23 05:13:11,Twitter Web App,@carmonliat מה חדש ?..שירת המקהלה.הם חושבים שא...,0.0,1405876438138605577,מיקי כהן,262,449,2766,[],1418434872939589633,3.115715e+09
53524,1418452815232610304,2021-07-23 06:07:59,Twitter Web App,@motiohana1 חושבים שאם יחוקקו חוק בזה תם עידן ...,0.0,1405876438138605577,מיקי כהן,262,449,2766,[],1418379856438571012,9.250476e+07
53525,1418472859886923776,2021-07-23 07:27:38,Twitter Web App,@yediotahronot @hadarse @gidonsaar @yuvalkarni...,0.0,1405876438138605577,מיקי כהן,262,449,2766,[],1418416831459692545,3.228184e+09
53526,1418473924225126408,2021-07-23 07:31:52,Twitter Web App,@ujcdmwtzdrooxwd חושבים שאם יחוקקו חוק בזה תם ...,0.0,1405876438138605577,מיקי כהן,262,449,2766,[],1418455910184259586,1.285077e+18


## Collecting tweets of new users to classify their tweets

In [ ]:
allTweetsPath = 'F:\\SocialDynamics\\TrendsProject\\Tweets\\New_Users_Tweets.csv'
users_list_path = 'F:\\SocialDynamics\\TrendsProject\\new_users_to_classify.txt'
files_dir = 'F:\\SocialDynamics\\TrendsProject\\Tweets\\NewUsersTweets'
read_and_get_users_tweets(allTweetsPath, users_list_path, files_dir, users_political_stance=0, with_header=False, sort_by='date')

In [ ]:
dir_path = 'F:\\SocialDynamics\\TrendsProject\\Tweets\\NewUsersTweets'
users_tweets_path = 'F:\\SocialDynamics\\TrendsProject\\Tweets\\New_Users_Tweets.csv'
read_jsons_and_write_to_csv(users_tweets_path, dir_path)

,tweet_id,created_at,source,text,retweet_count,user_id,user_name,user_followers_count,user_friends_count,user_statuses_count,hashtags,reply_to_status_id_str,reply_to_user_id,class
0,1388824691461197826,2021-05-02 11:56:24,Twitter for Android,@lamalo65903178 @riklin10 אחרי מה שכתבת אני הח...,0,849395682385506306,אדל,1423,1140,49701,[],1388823090860331016,1.312004e+18,0
1,1388825666922090499,2021-05-02 12:00:16,Twitter for Android,@lamalo65903178 @riklin10 שוב המנוול שופך זוהמ...,0,849395682385506306,אדל,1423,1140,49701,[],1388825373316685825,1.312004e+18,0
2,1388830211358302212,2021-05-02 12:18:20,Twitter for Android,@fpl_vicecaptain @avishaigrinzaig @riklin10 רי...,0,849395682385506306,אדל,1423,1140,49701,[],1388827959495114752,1.274235e+18,0
3,1388830616435793920,2021-05-02 12:19:56,Twitter for Android,@antoineking97 @zivfriedman יהודים ונוצרים מנש...,0,849395682385506306,אדל,1423,1140,49701,[],1388828795466047490,1.160347e+09,0
4,1388831028534468608,2021-05-02 12:21:35,Twitter for Android,@fpl_vicecaptain @avishaigrinzaig @riklin10 מי...,0,849395682385506306,אדל,1423,1140,49701,[],1388830628926398467,1.274235e+18,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20607,1419488420603342848,2021-07-26 02:43:06,Twitter for Android,@yoavgallant אתה באמת רוצה לספר לנו איך עושים ...,0,770830642661720064,Yossi Soffer,111,208,3563,[],1419338729022533633,8.249340e+17,1
20608,1419739768494112773,2021-07-26 19:21:52,Twitter for Android,@bardugojacob עלוב שכמוך. עדיין יש המון פחות ג...,0,770830642661720064,Yossi Soffer,111,208,3563,[],1419731221362397186,4.850518e+09,1
20609,1419739974799331329,2021-07-26 19:22:42,Twitter for Android,"@bittonrosen גם אין 6,400 מתים.",0,770830642661720064,Yossi Soffer,111,208,3563,[],1419572592193720322,1.058685e+18,1
20610,1419743884700229634,2021-07-26 19:38:14,Twitter for Android,@bardugojacob,0,770830642661720064,Yossi Soffer,111,208,3563,[],1419731221362397186,4.850518e+09,1
